In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

In [ ]:
'''
second performance test
'''
%time
import sys
sys.path.append('..')
%matplotlib inline
# you might need this:
#import matplotlib
#matplotlib.use('Agg')

from eden.converter.graph.gspan import gspan_to_eden
from graphlearn.graphlearn import GraphLearnSampler
import itertools
from eden.graph import Vectorizer
import matplotlib.pyplot as plt

from eden.util import fit_estimator,fit , random_bipartition_iter,estimate
import random
from eden.util import selection_iterator as picker
from sklearn.linear_model import SGDClassifier

vect=Vectorizer(complexity = 3)
#2401 positives and 1936 negatives
NUMPOS=2401
NUMNEG=1936

######################## testing ##############################

def train_estimator_and_evaluate_testsets(pos_original,neg_original,pos_augmented,neg_augmented,test_pos,test_neg):

    # create graph sets...orig augmented and sampled
    pos_orig,pos_orig_ = itertools.tee(pos_original,2)
    neg_orig,neg_orig_ = itertools.tee(neg_original,2)
    
    pos_augmented , pos_sampled = itertools.tee(pos_augmented)
    neg_augmented , neg_sampled = itertools.tee(neg_augmented)
    
    pos_augmented = itertools.chain(pos_augmented,pos_orig_)
    neg_augmented = itertools.chain(neg_augmented,neg_orig_)

    res=[]
    for desc,iterpos,iterneg in [('orignal',pos_orig, neg_orig),( 'original+sample',pos_augmented, neg_augmented),('sample',pos_sampled,neg_sampled)]:
        print 'working on %s'%desc
        test_pos,p=itertools.tee(test_pos)
        test_neg,n=itertools.tee(test_neg)
        res.append(estimate(p,n,fit(iterpos,iterneg,vect,n_jobs=-1),vect)[1])
    return res




def sample(graphs):
    sampler =GraphLearnSampler(radius_list=[1,2],thickness_list=[1,2])
    graphs, graphs_ = itertools.tee(graphs)
    sampler.fit(graphs,n_jobs=-1, core_interface_pair_remove_threshold=3, interface_remove_threshold=3)
    return sampler.sample(graphs_,
                        same_radius=False,
                        same_core_size=True,
                        sampling_interval=9999,
                        select_cip_max_tries=200,
                        batch_size=30,
                        n_steps=10,
                        n_jobs=-1,
                        annealing_factor=0.9)

In [ ]:
%time

# initializing 
graphs_pos= gspan_to_eden('bursi.pos.gspan')
graphs_neg= gspan_to_eden('bursi.neg.gspan')
originals=[]
improved=[]
only_new=[]

percentages=[.1, .2,.4,.6,.8,1]

for perc in percentages:

    ######### first we generate all the iterators ###########
    # copy the mega set
    graphs_pos, graphs_pos_ = itertools.tee(graphs_pos,2)
    graphs_neg, graphs_neg_ = itertools.tee(graphs_neg,2)


    # use shuffled list to create test and sample set
    pos,pos_test = random_bipartition_iter(graphs_pos_,0.7*perc)
    pos,pos_= itertools.tee(pos)
    neg,neg_test = random_bipartition_iter(graphs_neg_,0.7*perc)
    neg,neg_= itertools.tee(neg)
 
    ############### then we sample #####################
    sampled_neg= sample(neg)
    sampled_pos= sample(pos)
    
    
    ######### and last we evaluate ###########
    print "training perc:"+str(perc)
    print "*"*80
    aug,ori,new=train_estimator_and_evaluate_testsets( pos_,neg_,sampled_pos, sampled_neg, pos_test,neg_test)
    improved.append(aug)
    originals.append(ori)
    only_new.append(new)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


/Library/Python/2.7/site-packages/sklearn/calibration.py:398: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Library/Python/2.7/site-packages/sklearn/calibration.py:408: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Library/Python/2.7/site-packages/sklearn/calibration.py:410: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)


evaluating..
Test set
Instances: 3730 ; Features: 1048577 with an avg of 179 features per instance
--------------------------------------------------------------------------------
Test Estimate
             precision    recall  f1-score   support

         -1       0.74      0.77      0.75      1665
          1       0.81      0.78      0.79      2065

avg / total       0.78      0.78      0.78      3730

APR: 0.882
ROC: 0.857
select_cip_for_substitution failed because no suiting interface was found, extract failed 100 times 
select_cip_for_substitution failed because no suiting interface was found, extract failed 100 times 
select_cip_for_substitution failed because no suiting interface was found, extract failed 100 times 
propose failed.. reason is that propose_single_cip failed.
Test set
Instances: 3730 ; Features: 1048577 with an avg of 179 features per instance
--------------------------------------------------------------------------------
Test Estimate
             precision    

/Library/Python/2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


evaluating..
Test set
Instances: 3123 ; Features: 1048577 with an avg of 180 features per instance
--------------------------------------------------------------------------------
Test Estimate
             precision    recall  f1-score   support

         -1       0.78      0.82      0.80      1394
          1       0.85      0.81      0.83      1729

avg / total       0.82      0.81      0.81      3123

APR: 0.901
ROC: 0.884


In [ ]:
print 'original+sample:',improved
print 'original:',originals
print 'sample:',only_new

# draw 
plt.plot(originals ,'bs')
plt.plot(improved ,'r^')
plt.plot(only_new ,'gs')
plt.savefig('results.pdf')